# Случайный и направленный поиск лучших гиперпараметров.

Большинство моделей содержат гиперпараметры, требующие подбора для наилучшей точности, такие как тип ядра в SVM и степень регуляризации.

Как вы думаете, почему нельзя подбирать гиперпараметры по обучающей выборке, на которой настраивались обычные параметры моделей?

Случайный поиск лучше поиска по сетке за счет более широкого перебора по пространству **важных** гиперпараметров:

<img src="grid and random search.png" width="75%">

# Загрузка датасета

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import sklearn
from sklearn import datasets
import pandas as pd

In [3]:
data = datasets.fetch_openml(name='wdbc', version=1, parser='auto', data_home='datasets', 
                          as_frame=True)  

In [4]:
data.DESCR

'**Author**: William H. Wolberg, W. Nick Street, Olvi L. Mangasarian    \n**Source**: [UCI](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)), [University of Wisconsin](http://pages.cs.wisc.edu/~olvi/uwmp/cancer.html) - 1995  \n**Please cite**: [UCI](https://archive.ics.uci.edu/ml/citation_policy.html)     \n\n**Breast Cancer Wisconsin (Diagnostic) Data Set (WDBC).** Features are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. They describe characteristics of the cell nuclei present in the image. The target feature records the prognosis (benign (1) or malignant (2)). [Original data available here](ftp://ftp.cs.wisc.edu/math-prog/cpo-dataset/machine-learn/cancer/) \n\nCurrent dataset was adapted to ARFF format from the UCI version. Sample code ID\'s were removed.  \n\n! Note that there is also a related Breast Cancer Wisconsin (Original) Data Set with a different set of features, better known as [breast-w](https://www.openml.or

In [5]:
X=data.data
Y=data.target

In [6]:
X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [7]:
len(X), len(X.columns)

(569, 30)

In [8]:
Y.head()

0    2
1    2
2    2
3    2
4    2
Name: Class, dtype: category
Categories (2, object): ['1', '2']

In [9]:
Y=Y.cat.rename_categories([0,1]).astype(int)

In [10]:
Y.unique()

array([1, 0])

In [11]:
Z=X.copy()
Z['Y']=Y

In [12]:
np.random.seed(0)
Z = Z.sample(frac=1).reset_index(drop=True)

In [13]:
from sklearn.model_selection import train_test_split

Z_train, Z_test, ind_train, ind_test = train_test_split(Z, np.arange(len(Z)), test_size=0.4, shuffle=True, random_state=0, stratify=Z['Y'])   # разделим в на обучение/тест в заданной пропорции

len(Z), len(Z_train), len(Z_test)

(569, 341, 228)

In [14]:
def XY_split(Z):
    '''Функция разбиения по признаки и отклики'''
    Y = Z['Y']
    X = Z.copy()
    X = X.drop('Y',axis=1)
    return X,Y

X_train, Y_train = XY_split(Z_train)
X_test, Y_test = XY_split(Z_test)
X, Y = XY_split(Z)

Будем классифицировать объекты методом SVM.

Гиперпараметры:
- `C`: выбор между точностью и простотой модели
- `kernel in ['linear', 'poly', 'rbf']`: тип ядра
- `degree`: степень полиномиального ядра
- `rbf`=$\frac{1}{2 \sigma^2}$: контролирует разброс Гауссова ядра

In [15]:
from sklearn.svm import SVC

In [16]:
N_TRIALS = 40  # число испытаний (эффективнее, когда делится на #ядер процессора)

# Случайный поиск в `sklearn`

In [17]:
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import uniform, randint, loguniform

distributions = dict(C=loguniform(1e-10, 1e4),
                     kernel=['linear', 'poly', 'rbf'],
                     degree=randint(0, 5),
                     gamma=uniform(0, 2))

model = SVC()
clf = RandomizedSearchCV(model, distributions, n_iter=N_TRIALS, cv=[[ind_train, ind_test]], n_jobs=-1, verbose=10, random_state=0)

In [18]:
%time  search = clf.fit(X.to_numpy(), Y.to_numpy())
search.best_score_, search.best_params_

Fitting 1 folds for each of 40 candidates, totalling 40 fits
CPU times: user 1.12 s, sys: 84.7 ms, total: 1.21 s
Wall time: 42.7 s


(0.9473684210526315,
 {'C': 45.36050048212422,
  'degree': 0,
  'gamma': 1.957236684465528,
  'kernel': 'linear'})

Проверим инициализацию лучшими параметрами:

In [19]:
model = SVC(**search.best_params_)
model.fit(X_train, Y_train)
display(model)

accuracy = (model.predict(X_test) == Y_test).mean()
print(f'Accuracy: {accuracy:.3f}')

SVC(C=45.36050048212422, degree=0, gamma=1.957236684465528, kernel='linear')

Accuracy: 0.947


# Направленный поиск в Optuna

In [20]:
import optuna   # установка в conda: conda install -c conda-forge optuna

Теперь подбор гиперпараметров будем осуществлять с помощью библиотеки optuna.

Рассмотрим основные объекты:

Если нужно передавать дополнительные аргументы в `objective` - см. в [документации](https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).

In [21]:
# FYI: Objective functions can take additional arguments
# ().
def objective(trial):

    kernel = trial.suggest_categorical("kernel", ["linear", "poly", "rbf"])
    
    c = trial.suggest_float("C", 1e-10, 1e4, log=True)
    degree = 3
    gamma = 'scale'
    if kernel == "poly":
        degree = trial.suggest_int("degree", 0, 5)
    if kernel == "rbf":
        gamma = trial.suggest_float("gamma", 0, 2)
    
    model = SVC(C=c, kernel=kernel, degree=degree, gamma=gamma)
    
    model.fit(X_train, Y_train)
    accuracy = (model.predict(X_test) == Y_test).mean()

    return accuracy

Функция objective фактически должна содержать в себе весь пайплайн построения, обучения и применения модели, и должна возвращать некоторое число - нашу метрику оценивания модели. 
В качестве метрики можно использовать любое число.

Отличие objective от простого пайплайна модели в том, что эта функция принимает на вход аргумент trial, который можно использовать для получения **некоторых заранее неизвестных значений** гиперпараметров нашей модели из заранее определенного диапазона.

С помощью функций trial.suggest_int, trial.suggest_float и других мы можем получить некоторое значение параметра, которое используем дальше в пайплайне построения модели.

In [22]:
sampler = optuna.samplers.TPESampler(seed=0)  # Зафиксируем тип случайности для воспроизводимости. Необязательно в общем случае.
study = optuna.create_study(sampler=sampler, direction="maximize")

[I 2023-12-18 11:48:10,093] A new study created in memory with name: no-name-46a92068-68e4-467b-915d-3397d89af7b5


study - объект класса Study в библиотеке optuna -- определяет ход нашего "исследования".

Параметры позволяют определить, в какую сторону нужно оптимизировать метрику нашего objective, какой sampler и pruner использовать.

sampler - способ, которым будут выбираться параметры, получаемые с помощью trial.suggest_int и аналогичных функций. Базово используется байесовский TPESampler, подбирающий гиперпараметры по условным распределениям гиперпараметров. Также часто используются GridSampler - подбор параметров по сетке (декартово произведение задаваемых множеств параметров) - и RandomSampler - случайные значения параметров из заданных диапазонов.

"Исследование" запускается с помощью метода optimize:

In [23]:
%time study.optimize(objective, n_trials=N_TRIALS, n_jobs=-1) # n_jobs=-1 чтобы использовать все ядра
print(study.best_trial)

[I 2023-12-18 11:48:10,183] Trial 4 finished with value: 0.8421052631578947 and parameters: {'kernel': 'poly', 'C': 0.024026643899622033, 'degree': 1}. Best is trial 4 with value: 0.8421052631578947.
[I 2023-12-18 11:48:10,185] Trial 0 finished with value: 0.6271929824561403 and parameters: {'kernel': 'rbf', 'C': 185.65305870070313, 'gamma': 1.3597956021621178}. Best is trial 4 with value: 0.8421052631578947.
[I 2023-12-18 11:48:10,188] Trial 3 finished with value: 0.9342105263157895 and parameters: {'kernel': 'linear', 'C': 0.0003082417200598084}. Best is trial 3 with value: 0.9342105263157895.
[I 2023-12-18 11:48:10,212] Trial 2 finished with value: 0.6271929824561403 and parameters: {'kernel': 'poly', 'C': 7.758645312446727e-07, 'degree': 3}. Best is trial 3 with value: 0.9342105263157895.
[I 2023-12-18 11:48:10,219] Trial 6 finished with value: 0.6271929824561403 and parameters: {'kernel': 'poly', 'C': 2.3978944979632626e-09, 'degree': 1}. Best is trial 3 with value: 0.934210526315

CPU times: user 2min 20s, sys: 91.6 ms, total: 2min 20s
Wall time: 20.1 s
FrozenTrial(number=29, state=TrialState.COMPLETE, values=[0.9517543859649122], datetime_start=datetime.datetime(2023, 12, 18, 11, 48, 11, 40684), datetime_complete=datetime.datetime(2023, 12, 18, 11, 48, 11, 811843), params={'kernel': 'linear', 'C': 1.522362120911147}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'kernel': CategoricalDistribution(choices=('linear', 'poly', 'rbf')), 'C': FloatDistribution(high=10000.0, log=True, low=1e-10, step=None)}, trial_id=29, value=None)


Проверим инициализацию лучшими параметрами:

In [24]:
model = SVC(**study.best_params)
model.fit(X_train, Y_train)
display(model)
accuracy = (model.predict(X_test) == Y_test).mean()
print(f'Accuracy: {accuracy:.3f}')

SVC(C=1.522362120911147, kernel='linear')

Accuracy: 0.952


Видим, что времени на поиск затрачено меньше (`Wall time`), а результирующая точность (`Accuracy`) - выше.